<a href="https://colab.research.google.com/github/jmoranrun/HAR_Dist_ML/blob/main/Pruning_HAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pruning Experiments around Personalization for Human Actvity Recognition. 
I used the Karas Notebook "Magnitude-based weight pruning with Keras" as a reference.

This notebook was used to run the experiments in the presentation:
Human Activity Recognition in a Distributed Machine Learning System. Jul 26, 2021  IEEE International Conference on Wearable and Implantable Body Sensor Networks (BSN2021)





In [ ]:
! pip uninstall -y tensorflow
! pip uninstall -y tf-nightly
#! pip install -U tensorflow-gpu==1.14.0
! pip install -U tensorflow-gpu

! pip install tensorflow-model-optimization

In [ ]:
%load_ext tensorboard
import tensorboard
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow_model_optimization.sparsity import keras as sparsity
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D
import tensorflow as tf
import random 


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
import tensorflow as tf
#tf.enable_eager_execution()

import tempfile
import zipfile
import os

###Define functions
Define functions to allow the dataset to be manipulated at a per-user level of ganularity. 
Also a function is defined to count the number of weights that are set to zero. The purpose of this function is to confirm the sparsity of the model. </br>
This method was referenced from https://www.dlology.com/blog/how-to-compress-your-keras-model-x5-smaller-with-tensorflow-model-optimization/
Note, the method to move data from the test to training dataset is not currently used in these experiments 



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


 
# load a single file as a numpy array
def load_file(filepath):
  dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
  return dataframe.values
 

#######################################################################
# Function to move selected test samples to traning dataset
# Lots of parameters - however, these are driven by a higher level
# function (select_user_val), which has a more user friendly parameter list
####################################################################### 
def move_test_to_train(move_samples, move_sub_map, move_ys, move_idx, har_dataset_user_test, har_submap_user_test, har_dataset_user_train, har_submap_user_train, har_y_user_test, har_y_user_train):
    har_dataset_user_test = np.asarray(har_dataset_user_test)
    har_submap_user_test = np.asarray(har_submap_user_test)
    har_dataset_user_train = np.asarray(har_dataset_user_train)
    har_submap_user_train = np.asarray(har_submap_user_train)
    har_y_user_test = np.asarray(har_y_user_test)
    har_y_user_train = np.asarray(har_y_user_train)
    move_samples = np.asarray(move_samples)
    move_ys = np.asarray(move_ys)
    har_dataset_user_test = np.delete(har_dataset_user_test, move_idx, axis=0)
    har_submap_user_test = np.delete(har_submap_user_test, move_idx, axis=0)
    har_y_user_test = np.delete(har_y_user_test, move_idx, axis=0)
    har_dataset_user_train = np.concatenate((har_dataset_user_train, move_samples))
    har_submap_user_train = np.concatenate((har_submap_user_train, move_sub_map))
    har_y_user_train = np.concatenate((har_y_user_train, move_ys))
    return  har_dataset_user_test, har_submap_user_test, har_y_user_test, har_dataset_user_train, har_submap_user_train, har_y_user_train

#############################################################################################
# Function to create a user dataset
# har_dataset is a 3-D np array of form: [num_of_samples, num_of_timesamples, num_of_feature]
#             Where num_of_samples is the total number of recorded activites 
#                   num_of_timesamples is the total number of timesammples in each recorded activity 
#                   num_of_feature is the total number of features recorded using the smartphones.
# sub_map maps subjects (people) to each har_dataset sample
# test_user lists which subjects are assigned to the test dataset
# train_user lists which subjects are assigned to the training dataset
# percent_mix determines the percentage of the test dataset samples to move into the training dataset
#############################################################################################
def select_user_val(har_dataset, y, sub_map, test_users, train_user, percent_mix):
  har_dataset_user_test=[]
  har_submap_user_test=[]
  har_y_user_test=[]
  for user in test_users:
  #  har_dataset_user_test.append(har_dataset[tuple(np.where(sub_map==user))]) 
    har_dataset_user_test.extend(har_dataset[tuple(np.where(sub_map==user))].tolist()) 
    har_submap_user_test.extend(sub_map[np.where(sub_map==user)].tolist())
    har_y_user_test.extend(y[tuple(np.where(sub_map==user))].tolist()) 
  # Generate the default training dataset 
  har_dataset_user_train=[]
  har_submap_user_train=[]
  har_y_user_train=[]
  for user in train_user:
    har_dataset_user_train.extend(har_dataset[tuple(np.where(sub_map==user))].tolist())
    har_submap_user_train.extend(sub_map[np.where(sub_map==user)].tolist())
    har_y_user_train.extend(y[tuple(np.where(sub_map==user))].tolist()) 
 
  ## Now allow a percentage of val users samples to enter the training dataset
  ## Make sure that the percentage comes from each val user
 
  for user in test_users:
    har_user_sub_map=np.where(np.asarray(har_submap_user_test)==user)[0]
    har_user_sub_map_cnt=np.count_nonzero(np.asarray(har_submap_user_test)==user)
    num_take_sub_map = int(har_user_sub_map_cnt*percent_mix)
    if(num_take_sub_map > 0) :
      move_idx=random.sample(list(har_user_sub_map), num_take_sub_map)
      move_samples=[har_dataset_user_test[i] for i in move_idx]  
      move_sub_map=[har_submap_user_test[i] for i in move_idx]   
      move_ys=[har_y_user_test[i] for i in move_idx]   
      har_dataset_user_test, har_submap_user_test, har_y_user_test, har_dataset_user_train, har_submap_user_train, har_y_user_train \
        = move_test_to_train(move_samples, move_sub_map, move_ys, move_idx, har_dataset_user_test, har_submap_user_test, har_dataset_user_train, har_submap_user_train, har_y_user_test, har_y_user_train)
  return  np.asarray(har_dataset_user_test), np.asarray(har_submap_user_test), np.asarray(har_y_user_test), np.asarray(har_dataset_user_train), np.asarray(har_submap_user_train), np.asarray(har_y_user_train)
 
 

#https://www.dlology.com/blog/how-to-compress-your-keras-model-x5-smaller-with-tensorflow-model-optimization/ 

def check_zero_weights(model):
  #print(model.get_weights())
  for i, w in enumerate(model.get_weights()):
    print(
        "{} -- Total:{}, Zeros: {:.2f}%".format(
            model.weights[i].name, w.size, np.sum(w == 0) / w.size * 100
        )        
    )
    zero_idx=np.where(w == 0)
    print("i iddex is zero_idx is", i, zero_idx)  


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



### Define the CNN Model
In this cell the CNN is defined. </br>
It's initial weights are saved so that all experiments start from the same point to ensure fairness in the comparisions.


In [ ]:
##############################################
#### Define the CNN ##########################        
##############################################
# set variables
mix_users=True
seg_len = 128
num_channels = 9
num_labels = 6
batch_size = 100
learning_rate = 0.001
num_epoches = 0
#num_batches = train_x.shape[0] // batch_size
time_samples = 128
feature_sensors = 9
num_classes = 6


tf.compat.v1.disable_eager_execution()
l = tf.keras.layers

opt = 'sgd'
l = tf.keras.layers
model_har = tf.keras.Sequential([
    l.Conv1D(filters=32,kernel_size=10,strides=1,activation='relu', input_shape=(time_samples, feature_sensors)),
    l.MaxPooling1D(pool_size=4,strides=2, padding='same'),
    l.Conv1D(filters=64,kernel_size=2,strides=1,activation='relu'),
    l.MaxPooling1D(pool_size=4,strides=2, padding='same'),
    l.Conv1D(filters=128,kernel_size=2,strides=2,activation='relu'),
    l.Flatten(),
    l.Dense(150),
    l.Activation('tanh'),
    l.Dropout(0.5),
    l.Dense(50),
    l.Activation('tanh'),
    l.Dropout(0.5),
    l.Dense(25),
    l.Activation('tanh'),
    l.Dropout(0.5),
    l.Dense(num_labels),
    l.Activation('softmax')
])


model_har.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

## This NB needs to be run multiple times for one data collection due to 
## large computation effort.  Only write out the init weights once per
## data collection run to keep comparisions equal
gen_init_weights = False
if(gen_init_weights):
  model_har.save_weights('/content/drive/My Drive/model_har_pruning_init.h5')



print(model_har.summary())

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_42 (Conv1D)           (None, 119, 32)           2912      
_________________________________________________________________
max_pooling1d_28 (MaxPooling (None, 60, 32)            0         
_________________________________________________________________
conv1d_43 (Conv1D)           (None, 59, 64)            4160      
_________________________________________________________________
max_pooling1d_29 (MaxPooling (None, 30, 64)            0         
_________________________________________________________________
conv1d_44 (Conv1D)           (None, 15, 128)           16512     
_________________________________________________________________
flatten_14 (Flatten)         (None, 1920)              0         
_________________________________________________________________
dense_56 (Dense)             (None, 150)             

## Define The Pruning Flow
This method is the backbone of the pruning experiments. </br>
It creates edge and central datasets. </br>
Pruning at the central node can be enabled/disabled through switch prune_at_center </br>
Pruning at the edge nodes can be enabled/disabled through switch prune_at_edge </br>
Training at the edge nodes can be enabled/disabled through switch train_at_edge </br>
Training at the central nodes is always enabled </br>



In [ ]:
def run_prune(prune_at_edge, prune_at_center, train_at_edge, har_dataset, har_dataset_y, sub_map, edge_central_ratio, num_epoches = 25, eval_before_prune=False):

  total_num_users=30
  all_users = range(1, total_num_users)
  edge_users = random.sample(all_users, int(edge_central_ratio*total_num_users))
  #model_har.load_weights('/content/drive/My Drive/model_har_init.h5')
  central_users=np.setdiff1d(all_users, edge_users)
  print("Edge users are:",edge_users)
  print("Central users are:",central_users)

  percent_mix=0
  edge_x, edge_submap, edge_y, central_x, central_submap, central_y = \
        select_user_val(har_dataset, har_dataset_y, sub_map, edge_users, central_users, percent_mix)

  central_train_x, central_test_x, central_train_y, central_test_y = train_test_split(central_x, central_y, test_size = 0.2, random_state = 0,shuffle =True)
  edge_train_x, edge_test_x, edge_train_y, edge_test_y = train_test_split(edge_x, edge_y, test_size = 0.2, random_state = 0,shuffle =True)

  logdir = tempfile.mkdtemp()
  print('Writing training logs to ' + logdir)
  callbacks = [tf.keras.callbacks.TensorBoard(log_dir=logdir, profile_batch=0)]

  
  model_har.load_weights('/content/drive/My Drive/model_har_pruning_init.h5')
  if(eval_before_prune== True):
    model_har.fit(central_train_x, central_train_y,
           batch_size=batch_size,
           epochs=num_epoches,
           verbose=0,
           callbacks=callbacks,
           validation_data=(central_test_x, central_test_y))
    score = model_har.evaluate(central_test_x, central_test_y, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    check_zero_weights(model_har)

  num_train_samples = central_train_x.shape[0]
  end_step = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * num_epoches
  print('End step: ' + str(end_step))

  ## jmoran hand set the pruning paras

  if(prune_at_center==True):
   pruning_params = {
        'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.75,
                                                   begin_step=100,
                                                   end_step=end_step,
                                                   frequency=100)
    }
  else:
    pruning_params = {
        'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.0,
                                                   final_sparsity=0.0,
                                                   begin_step=100,
                                                   end_step=end_step,
                                                   frequency=100)
   }


  central_model_har = tf.keras.Sequential([
    sparsity.prune_low_magnitude(l.Conv1D(filters=32,kernel_size=10,strides=1,activation='relu', input_shape=(time_samples, feature_sensors)), **pruning_params),
    l.MaxPooling1D(pool_size=4,strides=2, padding='same'),
    sparsity.prune_low_magnitude(l.Conv1D(filters=64,kernel_size=2,strides=1,activation='relu'),**pruning_params),
    l.MaxPooling1D(pool_size=4,strides=2, padding='same'),
    sparsity.prune_low_magnitude(l.Conv1D(filters=128,kernel_size=2,strides=2,activation='relu'),**pruning_params),
    l.Flatten(),
    sparsity.prune_low_magnitude(l.Dense(150),**pruning_params),
    l.Activation('tanh'),
    l.Dropout(0.5),
    sparsity.prune_low_magnitude(l.Dense(50),**pruning_params),
    l.Activation('tanh'),
    l.Dropout(0.5),
    sparsity.prune_low_magnitude(l.Dense(25),**pruning_params),
    l.Activation('tanh'),
    l.Dropout(0.5),
    sparsity.prune_low_magnitude(l.Dense(num_labels),**pruning_params),
    l.Activation('softmax')
  ])

  #pruned_model_har.summary()
  central_model_har.compile(
     loss=tf.keras.losses.categorical_crossentropy,
     optimizer='adam',
     metrics=['accuracy'])

# Add a pruning step callback to peg the pruning step to the optimizer's
# step. Also add a callback to add pruning summaries to tensorboard
  callbacks = [
     sparsity.UpdatePruningStep(),
     sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
  ]

  central_model_har.fit(central_train_x, central_train_y,
           batch_size=batch_size,
           epochs=num_epoches,
           verbose=0,
           callbacks=callbacks,
           validation_data=(central_test_x, central_test_y))

  score = central_model_har.evaluate(central_test_x, central_test_y, verbose=0)
  print('Test loss:', score[0])
  print('Test accuracy:', score[1])
  check_zero_weights(central_model_har)
  central_model_har.save_weights('/content/drive/My Drive/pruned_model_har_central.h5')

  if(prune_at_edge==True):
   pruning_params_edge = {
         'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                    final_sparsity=0.75,
                                                     begin_step=100,
                                                    end_step=end_step,
                                                     frequency=100)
   }
  else:
   pruning_params_edge = {
         'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.0,
                                                    final_sparsity=0.0,
                                                    begin_step=100,
                                                    end_step=end_step,
                                                    frequency=100)
   }

  edge_pruned_model_har = tf.keras.Sequential([
     sparsity.prune_low_magnitude(l.Conv1D(filters=32,kernel_size=10,strides=1,activation='relu', input_shape=(time_samples, feature_sensors)), **pruning_params_edge),
     l.MaxPooling1D(pool_size=4,strides=2, padding='same'),
     sparsity.prune_low_magnitude(l.Conv1D(filters=64,kernel_size=2,strides=1,activation='relu'),**pruning_params_edge),
     l.MaxPooling1D(pool_size=4,strides=2, padding='same'),
     sparsity.prune_low_magnitude(l.Conv1D(filters=128,kernel_size=2,strides=2,activation='relu'),**pruning_params_edge),
     l.Flatten(),
      sparsity.prune_low_magnitude(l.Dense(150),**pruning_params_edge),
     l.Activation('tanh'),
     l.Dropout(0.5),
     sparsity.prune_low_magnitude(l.Dense(50),**pruning_params_edge),
     l.Activation('tanh'),
     l.Dropout(0.5),
     sparsity.prune_low_magnitude(l.Dense(25),**pruning_params_edge),
     l.Activation('tanh'),
     l.Dropout(0.5),
     sparsity.prune_low_magnitude(l.Dense(num_labels),**pruning_params_edge),
      l.Activation('softmax')
  ])

  edge_pruned_model_har.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

  if(prune_at_edge==True):          ## If Prune at edge take the model which has not had pruning done and apply the appriate weights from the central model (pruning can't be appied twice to a model)
   edge_pruned_model_har=edge_pruned_model_har
   edge_pruned_model_har.load_weights('/content/drive/My Drive/pruned_model_har_central.h5')
  else:
   edge_pruned_model_har=central_model_har

# Add a pruning step callback to peg the pruning step to the optimizer's
# step. Also add a callback to add pruning summaries to tensorboard
  callbacks = [
     sparsity.UpdatePruningStep(),
     sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
  ]

  if(train_at_edge==True):
   edge_pruned_model_har.fit(edge_train_x, edge_train_y,
           batch_size=batch_size,
           epochs=num_epoches,
           verbose=0,
           callbacks=callbacks,
           validation_data=(edge_test_x, edge_test_y))

  score = edge_pruned_model_har.evaluate(edge_test_x, edge_test_y, verbose=0)
  print('Edge Test loss:', score[0])
  print('Edge Test accuracy:', score[1])
  edge_acc =  score[1]

  score = central_model_har.evaluate(central_test_x, central_test_y, verbose=0)
  print('Central Test loss:', score[0])
  print('Central accuracy:', score[1])
  central_acc =  score[1]
  print(edge_pruned_model_har.summary())
  check_zero_weights(edge_pruned_model_har)

  return  central_acc, edge_acc 




### Prepare The Dataset and set the Run_prune parameters




In [ ]:

## This files contains the training users mapping
sub_map_train = load_file('/content/drive/My Drive/Colab Notebooks/UCI_dataset/UCI HAR Dataset/train/subject_train.txt')
train_subjects = np.unique(sub_map_train)
print(train_subjects)
 
sub_map_test = load_file('/content/drive/My Drive/Colab Notebooks/UCI_dataset/UCI HAR Dataset/test/subject_test.txt')
test_subjects = np.unique(sub_map_test)
print(test_subjects)
 
 
har_features = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]
 
num_of_timesamples = 128
num_of_feature = len(har_features)
num_of_samples = sub_map_train.shape[0] + sub_map_test.shape[0]
print("num_of_samples is",num_of_samples)

sub_map = np.concatenate((sub_map_test, sub_map_train), axis =0)
sub_map = sub_map.reshape(-1) 
print(sub_map)
unique, counts = np.unique(sub_map, return_counts=True)
print(dict(zip(unique, counts)))
 
har_feature_cnt=0
har_dataset =  np.empty([num_of_samples, num_of_timesamples, num_of_feature])
for har_feature in har_features:
   har_feature_test  = load_file(f'/content/drive/My Drive/Colab Notebooks/UCI_dataset/UCI HAR Dataset/test/Inertial Signals/{har_feature}_test.txt')
   har_feature_train = load_file(f'/content/drive/My Drive/Colab Notebooks/UCI_dataset/UCI HAR Dataset/train/Inertial Signals/{har_feature}_train.txt')
   har_feature = np.concatenate((har_feature_test, har_feature_train), axis=0)
   har_dataset[:, :, har_feature_cnt] = har_feature 
   har_feature_cnt += 1
 
har_dataset_with_subject = np.expand_dims(har_dataset, axis=1)
print("har_dataset_with_subject.shape is",har_dataset_with_subject.shape)
 
train_y =  load_file('/content/drive/My Drive/Colab Notebooks/UCI_dataset/UCI HAR Dataset/train/y_train.txt')
test_y =  load_file('/content/drive/My Drive/Colab Notebooks/UCI_dataset/UCI HAR Dataset/test/y_test.txt')
train_y = np.asarray(pd.get_dummies(train_y.reshape(-1, )))
test_y = np.asarray(pd.get_dummies(test_y.reshape(-1, )))
har_dataset_y = np.concatenate((test_y,train_y),axis=0)

num_runs_per_edge_ratio=20
edge_central_ratio_lst = [0.4, 0.5, 0.6, 0.7, 0.8]



[ 1  3  5  6  7  8 11 14 15 16 17 19 21 22 23 25 26 27 28 29 30]
[ 2  4  9 10 12 13 18 20 24]
num_of_samples is 10299
[ 2  2  2 ... 30 30 30]
{1: 347, 2: 302, 3: 341, 4: 317, 5: 302, 6: 325, 7: 308, 8: 281, 9: 288, 10: 294, 11: 316, 12: 320, 13: 327, 14: 323, 15: 328, 16: 366, 17: 368, 18: 364, 19: 360, 20: 354, 21: 408, 22: 321, 23: 372, 24: 381, 25: 409, 26: 392, 27: 376, 28: 382, 29: 344, 30: 383}
har_dataset_with_subject.shape is (10299, 1, 128, 9)


### Run Pruning experiment with: <br>
prune_at_edge=False<br>
prune_at_center=False<br>
train_at_edge=False<br> 
Results will be printed to file 



In [ ]:
prune_at_edge=False
prune_at_center=False
train_at_edge=False


central_acc_lst=[]
edge_acc_lst=[]

for edge_central_ratio in edge_central_ratio_lst:
  for run in range(num_runs_per_edge_ratio):
    central_acc, edge_acc = run_prune(prune_at_edge=prune_at_edge, prune_at_center=prune_at_center, train_at_edge=train_at_edge, har_dataset=har_dataset, har_dataset_y=har_dataset_y, sub_map=sub_map, edge_central_ratio=edge_central_ratio)
    central_acc_lst.append(central_acc)
    edge_acc_lst.append(edge_acc)

with open('/content/drive/My Drive/pef_pcf_tef_bl_20.txt', 'w') as f:
        f.writelines("%s\n" % central_acc_lst)
        f.writelines("%s\n" % edge_acc_lst)   
print("central_acc_lst is", central_acc_lst)
print("edge_acc_lst is",edge_acc_lst)


### Run Pruning experiment with: <br>
prune_at_edge=False<br>
prune_at_center=True<br>
train_at_edge=False<br> 
Results will be printed to file

In [ ]:

prune_at_edge=False
prune_at_center=True
train_at_edge=False

central_acc_lst1=[]
edge_acc_lst1=[]


for edge_central_ratio in edge_central_ratio_lst:
  for run in range(num_runs_per_edge_ratio):
    central_acc, edge_acc = run_prune(prune_at_edge=prune_at_edge, prune_at_center=prune_at_center, train_at_edge=train_at_edge, har_dataset=har_dataset, har_dataset_y=har_dataset_y, sub_map=sub_map, edge_central_ratio=edge_central_ratio)
    central_acc_lst1.append(central_acc)
    edge_acc_lst1.append(edge_acc)

with open('/content/drive/My Drive/pef_pct_tef_bl_20.txt', 'w') as f:
        f.writelines("%s\n" % central_acc_lst1)
        f.writelines("%s\n" % edge_acc_lst1)   

print("central_acc_lst is", central_acc_lst1)
print("edge_acc_lst is",edge_acc_lst1)

### Run Pruning experiment with: <br>
prune_at_edge=False<br>
prune_at_center=True<br>
train_at_edge=True<br> 
Results will be printed to file

In [ ]:

prune_at_edge=False
prune_at_center=True
train_at_edge=True

central_acc_lst2=[]
edge_acc_lst2=[]

for edge_central_ratio in edge_central_ratio_lst:
  for run in range(num_runs_per_edge_ratio):
    central_acc, edge_acc = run_prune(prune_at_edge=prune_at_edge, prune_at_center=prune_at_center, train_at_edge=train_at_edge, har_dataset=har_dataset, har_dataset_y=har_dataset_y, sub_map=sub_map, edge_central_ratio=edge_central_ratio)
    central_acc_lst2.append(central_acc)
    edge_acc_lst2.append(edge_acc)

with open('/content/drive/My Drive/pef_pct_tet_bl_20.txt', 'w') as f:
        f.writelines("%s\n" % central_acc_lst2)
        f.writelines("%s\n" % edge_acc_lst2)   

print("central_acc_lst is", central_acc_lst2)
print("edge_acc_lst is",edge_acc_lst2)

### Run Pruning experiment with: <br>
prune_at_edge=True<br>
prune_at_center=False<br>
train_at_edge=True<br> 
Results will be printed to file

In [ ]:

prune_at_edge=True
prune_at_center=False
train_at_edge=True

central_acc_lst3=[]
edge_acc_lst3=[]

for edge_central_ratio in edge_central_ratio_lst:
  for run in range(num_runs_per_edge_ratio):
    central_acc, edge_acc = run_prune(prune_at_edge=prune_at_edge, prune_at_center=prune_at_center, train_at_edge=train_at_edge, har_dataset=har_dataset, har_dataset_y=har_dataset_y, sub_map=sub_map, edge_central_ratio=edge_central_ratio)
    central_acc_lst3.append(central_acc)
    edge_acc_lst3.append(edge_acc)

with open('/content/drive/My Drive/pet_pcf_tet_bl_20.txt', 'w') as f:
        f.writelines("%s\n" % central_acc_lst3)
        f.writelines("%s\n" % edge_acc_lst3)   

print("central_acc_lst is", central_acc_lst3)
print("edge_acc_lst is",edge_acc_lst3)